In [1]:
# importing necessary libraries

import pandas as pd
import requests
import json
import time
from datetime import datetime
import pickle
import psycopg2     
from psycopg2 import sql
from dotenv import load_dotenv 
import os
import logging
import great_expectations as ge
# import kafka
# from kafka import KafkaProducer


In [2]:
import great_expectations as ge
from great_expectations.validator.validator import Validator
from great_expectations.execution_engine.pandas_execution_engine import PandasExecutionEngine
from great_expectations.execution_engine import PandasExecutionEngine
from great_expectations.core.batch import Batch
from great_expectations.core import ExpectationSuite
from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations import get_context
from great_expectations.core.batch import Batch
from great_expectations.execution_engine import PandasExecutionEngine
from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations import get_context
import tempfile

Data Extraction Layer

In [3]:
# Load environment variables
# def data_ingestion_layer(**kwargs):
#     load_dotenv()
    
app_id = os.getenv('API_ID')
app_key = os.getenv('API_KEY')
POLL_INTERVAL = 60  # Time in seconds between requests

# Define the endpoint and parameters for scheduled data
station_code = 'RMD'  # Example station code (London Waterloo)
url = f'https://transportapi.com/v3/uk/train/station/{station_code}/live.json'

params = {
    'app_id': app_id,
    'app_key': app_key,
    #'time_of_day': '19:00',
    #'request_time': '2024-10-30T18:50:00+00:00',
    'darwin': 'false',  
    'train_status': 'passenger',  # Status filter, e.g., passenger trains only
    'live' :'True',
    #'station_detail': 'destination'
}
response = requests.get(url, params=params)

if response.status_code == 200:
    response= response.json()
    #kwargs['ti'].xcom_push(key='raw_response', value=response_json)
    #producer.send('train_departures', {'data': response})
    print("Data successfully extracted for train_departures")
else:
    print(f"Failed to fetch data: {response.status_code}")
    print(response.text)

Data successfully extracted for train_departures


In [4]:
response

{'date': '2024-12-09',
 'time_of_day': '13:29',
 'request_time': '2024-12-09T13:29:19+00:00',
 'station_name': 'Richmond',
 'station_code': 'RMD',
 'departures': {'all': [{'mode': 'train',
    'service': '22214000',
    'train_uid': 'P90715',
    'platform': '6',
    'operator': 'LO',
    'operator_name': 'London Overground',
    'aimed_departure_time': '13:31',
    'aimed_arrival_time': None,
    'aimed_pass_time': None,
    'origin_name': 'Richmond',
    'destination_name': 'Stratford',
    'source': 'Network Rail',
    'category': 'OO',
    'service_timetable': {'id': 'https://transportapi.com/v3/uk/train/service_timetables/P90715:2024-12-09.json?app_id=4f292769&app_key=3e6756c7b1419aaf07d4370c42cb58d2&live=true'},
    'status': 'STARTS HERE',
    'expected_arrival_time': None,
    'expected_departure_time': '13:31',
    'best_arrival_estimate_mins': None,
    'best_departure_estimate_mins': 1},
   {'mode': 'train',
    'service': '24682004',
    'train_uid': 'C13699',
    'platform

Transformation

In [5]:
with open("response.pkl", "wb") as f:
    pickle.dump(response, f)

vtrain_columns= []




request_time = response['request_time']
station_name = response['station_name']

train_info = {
    
    'request_time': request_time,
    'station_name': station_name
}
vtrain_columns.append(train_info)

vtrain_columns



vtrain_columns_df = pd.DataFrame(vtrain_columns)
vtrain_columns_df.reset_index(inplace=True)
vtrain_columns_df




departure_columns = []

for columns in response['departures']['all']:
    try:
        row ={
            'mode':columns['mode'],
            'train_uid': columns['train_uid'],
            'origin_name': columns['origin_name'],
            'operator_name':columns['operator_name'],
            'platform': columns['platform'],
            'destination_name': columns['destination_name'],
            'aimed_departure_time': columns['aimed_departure_time'],
            'expected_departure_time': columns['expected_departure_time'],
            'best_departure_estimate_mins': columns['best_departure_estimate_mins'],
            'aimed_arrival_time': columns['aimed_arrival_time']
        }
        departure_columns.append( row)
    except (KeyError, TypeError) as e:
        print(f"Error processing columns: {e}")     

print(departure_columns)




departure_columns_df = pd.DataFrame(departure_columns)
departure_columns_df.reset_index(inplace=True)
departure_columns_df

[{'mode': 'train', 'train_uid': 'P90715', 'origin_name': 'Richmond', 'operator_name': 'London Overground', 'platform': '6', 'destination_name': 'Stratford', 'aimed_departure_time': '13:31', 'expected_departure_time': '13:31', 'best_departure_estimate_mins': 1, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'C13699', 'origin_name': 'Upminster Underground', 'operator_name': 'London Underground', 'platform': '7', 'destination_name': 'Upminster Underground', 'aimed_departure_time': '13:34', 'expected_departure_time': '13:33', 'best_departure_estimate_mins': 3, 'aimed_arrival_time': '13:22'}, {'mode': 'train', 'train_uid': 'L58571', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '1', 'destination_name': 'Reading', 'aimed_departure_time': '13:36', 'expected_departure_time': '13:35', 'best_departure_estimate_mins': 5, 'aimed_arrival_time': '13:36'}, {'mode': 'train', 'train_uid': 'C13709', 'origin_name': 'Upminster Underground', 'operator_

,index,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,train,P90715,Richmond,London Overground,6,Stratford,13:31,13:31,1.0,None
1,1,train,C13699,Upminster Underground,London Underground,7,Upminster Underground,13:34,13:33,3.0,13:22
2,2,train,L58571,London Waterloo,South Western Railway,1,Reading,13:36,13:35,5.0,13:36
3,3,train,C13709,Upminster Underground,London Underground,6,Upminster Underground,13:43,13:43,13.0,13:34
4,4,train,L58576,Reading,South Western Railway,2,London Waterloo,13:45,13:45,15.0,13:45
5,5,train,P90719,Richmond,London Overground,4,Stratford,13:47,13:47,17.0,None
6,6,train,C13724,Upminster Underground,London Underground,5,Upminster Underground,13:53,13:53,23.0,13:42
7,7,train,L62096,London Waterloo,South Western Railway,1,Windsor & Eton Riverside,13:53,13:53,23.0,13:52
8,8,train,L61313,London Waterloo,South Western Railway,1,London Waterloo,13:58,13:58,28.0,13:58
9,9,train,L62100,Windsor & Eton Riverside,South Western Railway,2,London Waterloo,13:58,13:57,27.0,13:57


In [6]:
# Merging both dataframes

raw_schedule_df = pd.merge(vtrain_columns_df, departure_columns_df, on='index', how='outer' )
raw_schedule_df

,index,request_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-12-09T13:29:19+00:00,Richmond,train,P90715,Richmond,London Overground,6,Stratford,13:31,13:31,1.0,None
1,1,NaN,NaN,train,C13699,Upminster Underground,London Underground,7,Upminster Underground,13:34,13:33,3.0,13:22
2,2,NaN,NaN,train,L58571,London Waterloo,South Western Railway,1,Reading,13:36,13:35,5.0,13:36
3,3,NaN,NaN,train,C13709,Upminster Underground,London Underground,6,Upminster Underground,13:43,13:43,13.0,13:34
4,4,NaN,NaN,train,L58576,Reading,South Western Railway,2,London Waterloo,13:45,13:45,15.0,13:45
5,5,NaN,NaN,train,P90719,Richmond,London Overground,4,Stratford,13:47,13:47,17.0,None
6,6,NaN,NaN,train,C13724,Upminster Underground,London Underground,5,Upminster Underground,13:53,13:53,23.0,13:42
7,7,NaN,NaN,train,L62096,London Waterloo,South Western Railway,1,Windsor & Eton Riverside,13:53,13:53,23.0,13:52
8,8,NaN,NaN,train,L61313,London Waterloo,South Western Railway,1,London Waterloo,13:58,13:58,28.0,13:58
9,9,NaN,NaN,train,L62100,Windsor & Eton Riverside,South Western Railway,2,London Waterloo,13:58,13:57,27.0,13:57


In [7]:
#now lets rename some of the columns
train_schedule_df = raw_schedule_df

#Renaming request time column
train_schedule_df.rename(columns= {'request_time':'request_date_time'}, inplace = True)
train_schedule_df['request_date_time'] = train_schedule_df['request_date_time'].ffill()
train_schedule_df['station_name'] = train_schedule_df['station_name'].ffill()

# Fill all Null values 
train_schedule_df.fillna('unknown', inplace=True)

#Dropping the index column
train_schedule_df.drop( axis=0, columns='index', level=None, inplace=True, errors='raise')



/tmp/ipykernel_217728/1797525534.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train_schedule_df.fillna('unknown', inplace=True)


In [8]:
train_schedule_df

,request_date_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,2024-12-09T13:29:19+00:00,Richmond,train,P90715,Richmond,London Overground,6,Stratford,13:31,13:31,1.0,unknown
1,2024-12-09T13:29:19+00:00,Richmond,train,C13699,Upminster Underground,London Underground,7,Upminster Underground,13:34,13:33,3.0,13:22
2,2024-12-09T13:29:19+00:00,Richmond,train,L58571,London Waterloo,South Western Railway,1,Reading,13:36,13:35,5.0,13:36
3,2024-12-09T13:29:19+00:00,Richmond,train,C13709,Upminster Underground,London Underground,6,Upminster Underground,13:43,13:43,13.0,13:34
4,2024-12-09T13:29:19+00:00,Richmond,train,L58576,Reading,South Western Railway,2,London Waterloo,13:45,13:45,15.0,13:45
5,2024-12-09T13:29:19+00:00,Richmond,train,P90719,Richmond,London Overground,4,Stratford,13:47,13:47,17.0,unknown
6,2024-12-09T13:29:19+00:00,Richmond,train,C13724,Upminster Underground,London Underground,5,Upminster Underground,13:53,13:53,23.0,13:42
7,2024-12-09T13:29:19+00:00,Richmond,train,L62096,London Waterloo,South Western Railway,1,Windsor & Eton Riverside,13:53,13:53,23.0,13:52
8,2024-12-09T13:29:19+00:00,Richmond,train,L61313,London Waterloo,South Western Railway,1,London Waterloo,13:58,13:58,28.0,13:58
9,2024-12-09T13:29:19+00:00,Richmond,train,L62100,Windsor & Eton Riverside,South Western Railway,2,London Waterloo,13:58,13:57,27.0,13:57


In [10]:
#Saving file to CSV
train_schedule_df.to_csv('velorail_schedule.csv')

Great Expectation Validation

In [ ]:
#validating transformed data

try:
    context = get_context()


    suite = ExpectationSuite("train_schedule_suite")


    # Set up an execution engine
    execution_engine = PandasExecutionEngine()

    # Use a Batch to wrap the DataFrame
    batch = Batch(data=train_schedule_df)

    # Step 3: Create an Expectation Suite
   

    # Create a Validator with the Batch and ExpectationSuite
    validator = Validator(
        execution_engine=execution_engine,
        batches=[batch],
        expectation_suite=suite
    )

    # Add expectations directly to the Validator
    validator.expect_column_values_to_not_be_null(column="request_date_time")
    validator.expect_column_values_to_not_be_null(column="station_name")
    validator.expect_column_values_to_not_be_null(column="mode")
    validator.expect_column_values_to_not_be_null(column="train_uid")
    validator.expect_column_values_to_not_be_null(column="origin_name")
    validator.expect_column_values_to_not_be_null(column="operator_name")
    validator.expect_column_values_to_not_be_null(column="platform")
    validator.expect_column_values_to_not_be_null(column="destination_name")
    validator.expect_column_values_to_not_be_null(column="aimed_departure_time")
    validator.expect_column_values_to_not_be_null(column="expected_departure_time")
    validator.expect_column_values_to_not_be_null(column="best_departure_estimate_mins")
    validator.expect_column_values_to_not_be_null(column="aimed_arrival_time") 

    # Step 6: Validate the DataFrame and print the results
    transformed_results = validator.validate()
    print("Validation results:", transformed_results)

except Exception as e:
    print(f"Data Validation failed: {e}")

/mnt/c/Users/Josey/velocityrailways/railenv/lib/python3.10/site-packages/great_expectations/expectations/expectation.py:1456: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 13.96it/s]
/mnt/c/Users/Josey/velocityrailways/railenv/lib/python3.10/site-packages/great_expectations/expectations/expectation.py:1456: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 21.34it/s]
/mnt/c/Users/Josey/velocityrailways/railenv/lib/python3.10/site-packages/great_expectations/expectations/expectation.py:1456: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the

Validation results: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "request_date_time"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "station_name"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_inf

Data Loading Into Our Local Database

In [13]:
import dotenv
import os
import psycopg2

# Load environment variables
dotenv.load_dotenv(override=True)

# Function to establish a database connection
def get_connection(DB_host, dbname, user, password, port):
    return psycopg2.connect(
        host='localhost',
        dbname= 'velocityrailway_db_cron',
        user= 'joanna',
        password= 'Avokerie3',
        port=5432
    )

# Function to close connections
def close_connection(conn):
    try:
        if conn:
            conn.close()
            print(f"Closed connection to database")
    except Exception as e:
        print(f"Error closing connection: {e}")

# SQL Queries
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS train_schedule (
    id SERIAL PRIMARY KEY,
    request_date_time VARCHAR(100),
    station_name VARCHAR(100),
    mode VARCHAR(100),
    train_uid VARCHAR(100),
    origin_name VARCHAR(100),
    operator_name VARCHAR(100),
    platform VARCHAR(100),
    destination_name VARCHAR(100),
    aimed_departure_time VARCHAR(100),
    expected_departure_time VARCHAR(100),
    best_departure_estimate_mins VARCHAR(100),
    aimed_arrival_time VARCHAR(100)
);
"""

INSERT_SQL = """
INSERT INTO train_schedule (
    request_date_time, station_name, mode, train_uid, origin_name, operator_name, 
    platform, destination_name, aimed_departure_time, expected_departure_time, 
    best_departure_estimate_mins, aimed_arrival_time
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Function to load and prepare data from CSV
def load_data_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    # Ensure columns match the table definition
    expected_columns = [
        "request_date_time", "station_name", "mode", "train_uid", "origin_name", 
        "operator_name", "platform", "destination_name", "aimed_departure_time", 
        "expected_departure_time", "best_departure_estimate_mins", "aimed_arrival_time"
    ]
    df = df[expected_columns]
    return [tuple(row) for row in df.values]

# Function to create table and insert data
def store_data(data):
    connections = [
        get_connection(os.getenv("DB_HOST"), os.getenv("DB_NAME"), os.getenv("DB_USER"), os.getenv("DB_PASSWORD"), os.getenv("DB_PORT")),
        get_connection(os.getenv("AZURE_DB_HOST"), os.getenv("AZURE_DB_NAME"), os.getenv("AZURE_DB_USER"), os.getenv("AZURE_DB_PASSWORD"), os.getenv("AZURE_DB_PORT"))
    ]

    for conn in connections:
        try:
            with conn.cursor() as cur:
                # Create table
                print(f"Table created in database")
                cur.execute(CREATE_TABLE_SQL)
                conn.commit()

                # Insert data
                print(f"Data inserted into database")
                cur.executemany(INSERT_SQL, data)
                conn.commit()

                
        except Exception as e:
            print(f"Error in {conn.dsn}: {e}")
            conn.rollback()
        finally:
            close_connection(conn)

# Load data and store in databases
csv_path = "train_schedule.csv"
validated_data = load_data_from_csv(csv_path)
store_data(validated_data)

Table created in database
Data inserted into database
Closed connection to database
Table created in database
Data inserted into database
Closed connection to database


Ochestrated Airflow DAG

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from datetime import timedelta
import os

# Import functions from the script
from etl_pipeline import (
    data_ingestion_layer,
    data_transformation_layer,
    data_validation_layer,
    data_duplication_layer
)

# Default DAG arguments
default_args = {
    'owner': 'airflow',
    'depends_on_past': True,
    'email': 'ijonijosey@gmail.com',
    'email_on_failure': True,
    'email_on_retry': True,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
with DAG(
    'train_data_pipeline',
    default_args=default_args,
    description='A DAG for ingesting, transforming, validating, and duplicating train schedule data',
    schedule_interval=timedelta(hours=1),  # Adjust schedule as needed
    start_date=days_ago(1),
    catchup=False,
) as dag:

    # Task 1: Data Ingestion
    ingest_task = PythonOperator(
        task_id='ingest_data',
        python_callable=data_ingestion_layer
    )

    # Task 2: Data Transformation
    transform_task = PythonOperator(
        task_id='transform_data',
        python_callable=data_transformation_layer
    )

    # Task 3: Data Validation
    validate_task = PythonOperator(
        task_id='validate_data',
        python_callable=data_validation_layer
    )

    # Task 4: Data Duplication
    duplicate_task = PythonOperator(
        task_id='duplicate_data',
        python_callable=data_duplication_layer,
        op_kwargs={'csv_path': os.path.join(os.getcwd(), 'train_schedule.csv')}
    )

    # Define task dependencies
    ingest_task >> transform_task >> validate_task >> duplicate_task

ImportError: cannot import name 'data_ingestion_layer' from 'etl_pipeline' (/mnt/c/Users/Josey/velocityrailways/etl_pipeline.py)